# 02 - Submiting the Run with Various Compute Options

In [ ]:
import environs

e_vars = environs.Env()
e_vars.read_env('../workshop.env')

USER_NAME = e_vars.str("USER_NAME")
EXPERIMENT_NAME = e_vars.str('EXPERIMENT_NAME')
ENVIRONMENT_NAME = e_vars.str("ENVIRONMENT_NAME")
DATASET_NAME = e_vars.str("DATASET_NAME")

CLUSTER_NAME = e_vars.str("CLUSTER_NAME")

if not USER_NAME:
    raise NotImplementedError("Please enter your username in the `.env` file and run this cell again.")

In [ ]:
from azureml.core import Workspace, Experiment, Run, Environment, Dataset
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling, uniform, choice

In [ ]:
ws = Workspace.from_config()

# Attach to experiment
experiment = Experiment(workspace=ws, name=EXPERIMENT_NAME)

## Retrieve Environment, Compute Target, and Dataset Objects

In [ ]:
environ = ws.environments[ENVIRONMENT_NAME]
dataset = ws.datasets[DATASET_NAME]
cpu_cluster = ws.compute_targets[CLUSTER_NAME]

---
# Submit Run using Docker Environment on 'Local' Compute

In [ ]:
args = {"--dataset-name": 'input_dataset'}

local_est = Estimator(source_directory=".",
                compute_target='local',
                entry_script="train.py",
                script_params=args,
                environment_definition=environ,
                inputs=[dataset.as_named_input('input_dataset')]
                )

In [ ]:
local_run = experiment.submit(local_est, tags={'Context': "Local Compute"})

In [ ]:
_ = local_run.wait_for_completion(show_output=True)

---
# Submit to AML Compute for Computation

In [ ]:
args = {"--dataset-name": 'input_dataset'}

est = Estimator(source_directory=".",
                compute_target=cpu_cluster,
                entry_script="train.py",
                script_params=args,
                environment_definition=environ,
                inputs=[dataset.as_named_input('input_dataset')]
                )

In [ ]:
run = experiment.submit(est, tags={'Context': "AML Compute"})

In [ ]:
_ = run.wait_for_completion(show_output=True)

---
# Submit to AML Compute with Hyperparameter Sampling

In [ ]:
sampled_params = RandomParameterSampling({
                        '--alpha': uniform(0, 1),
                        '--l1-ratio': uniform(0, 1),
                        '--model-name': choice('ridge', 'elastic', 'gbt'),
                        '--learning-rate': uniform(0, 1)
                        })

hyper_run_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=sampled_params,
                                    primary_metric_name='rmse',
                                    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                    max_total_runs=10,
                                    max_concurrent_runs=2)

In [ ]:
hd_run = experiment.submit(hyper_run_config, tags={"Context": "Hyperdrive"})

In [ ]:
_ = hd_run.wait_for_completion(show_output=True)

<br><br><br><br><br>






###### Copyright (c) Microsoft Corporation. All rights reserved.  
###### Licensed under the MIT License.